In [2]:
import os  
import sys  
import zipfile  
import tempfile  
import shutil  
from tqdm.notebook import tqdm  
from dotenv import load_dotenv  

# 프로젝트 루트 경로 설정  
#sys.path.append('./src')  


from update_objectfile import upload_file_with_metadata  
from data_engineering.obj_storage.boto3 import MinIOClient  
from data_engineering.db.connector import Database  

In [3]:
# 환경 변수 로드  
load_dotenv('.env')  

# MinIO 클라이언트 초기화  
minio_client = MinIOClient()  

# 데이터셋 경로 설정 (상대 경로 사용)  
DATASET_PATH = "../../DataSet"  

# 처리할 디렉토리 구조  
DIRECTORIES = [  
    "Training/01.원천데이터",  
    "Training/02.라벨링데이터",  
    "Validation/01.원천데이터",  
    "Validation/02.라벨링데이터"  
]  

In [3]:
# 버킷 이름 설정  
BUCKET_NAME = "tada"  

# 지원하는 이미지 확장자  
IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".gif", ".bmp"}  

def is_image_file(filename):  
    return any(filename.lower().endswith(ext) for ext in IMAGE_EXTENSIONS)  

def get_uploaded_files():  
    """DB에서 이미 업로드된 파일 목록 가져오기"""  
    db = Database(  
        host=os.getenv('DB_HOST'),  
        port=int(os.getenv('DB_PORT')),  
        user=os.getenv('DB_USER'),  
        password=os.getenv('DB_PASSWORD'),  
        database=os.getenv('DB_NAME')  
    )  
    
    query = "SELECT category, object_path FROM object_storage"  
    results = db.getter(query)  
    return {(record['category'], record['object_path']) for record in results}  

In [ ]:
# DB 테이블 생성 확인  
from data_engineering.db.init_db import create_object_storage_table  
create_object_storage_table()  

# 이미 업로드된 파일 목록 가져오기  
uploaded_files = get_uploaded_files()  
print(f"이미 업로드된 파일 수: {len(uploaded_files)}")  

# 임시 디렉토리 생성  
temp_dir = tempfile.mkdtemp()  
print(f"임시 디렉토리 생성됨: {temp_dir}")  

try:  
    # 각 디렉토리 처리  
    for dir_path in DIRECTORIES:  
        current_path = os.path.join(DATASET_PATH, dir_path)  
        print(f"\n처리 중인 디렉토리: {dir_path}")  
        
        # ZIP 파일 목록 가져오기  
        zip_files = [f for f in os.listdir(current_path) if f.endswith('.zip')]  
        print(f"발견된 ZIP 파일 수: {len(zip_files)}")  
        
        # 각 ZIP 파일 처리  
        for zip_filename in tqdm(zip_files, desc="ZIP 파일 처리 중"):  
            zip_path = os.path.join(current_path, zip_filename)  
            category = f"{dir_path}/{os.path.splitext(zip_filename)[0]}"  
            
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:  
                for file_info in tqdm(zip_ref.filelist, desc="파일 추출 중"):  
                    if is_image_file(file_info.filename):  
                        object_key = f"{category}/{os.path.basename(file_info.filename)}"  
                        
                        # 이미 업로드된 파일 건너뛰기  
                        if (category, object_key) in uploaded_files:  
                            continue  
                        
                        # 임시 파일로 추출  
                        temp_path = os.path.join(temp_dir, os.path.basename(file_info.filename))  
                        with zip_ref.open(file_info) as source, open(temp_path, 'wb') as target:  
                            shutil.copyfileobj(source, target)  
                        
                        try:  
                            # MinIO 업로드 및 메타데이터 저장  
                            success = upload_file_with_metadata(  
                                local_path=temp_path,  
                                category=category,  
                                bucket=BUCKET_NAME,  
                                object_path=object_key  
                            )  
                            if not success:  
                                print(f"업로드 실패: {object_key}")  
                        except Exception as e:  
                            print(f"처리 실패: {object_key} - {str(e)}")  
                        
                        # 임시 파일 삭제  
                        os.remove(temp_path)  

finally:  
    # 임시 디렉토리 정리  
    shutil.rmtree(temp_dir)  
    print("임시 디렉토리 정리 완료")  

object_storage 테이블이 성공적으로 생성되었습니다.
이미 업로드된 파일 수: 13838
임시 디렉토리 생성됨: C:\Users\dongr\AppData\Local\Temp\tmpc16pon_w

처리 중인 디렉토리: Training/01.원천데이터
발견된 ZIP 파일 수: 8


ZIP 파일 처리 중:   0%|          | 0/8 [00:00<?, ?it/s]

파일 추출 중:   0%|          | 0/6058 [00:00<?, ?it/s]

Upload error: Read timeout on endpoint URL: "http://175.214.62.133:50008/tada/Training/01.%EC%9B%90%EC%B2%9C%EB%8D%B0%EC%9D%B4%ED%84%B0/TS_OBJ_1/APT_FP_OBJ_037344144.PNG?uploadId=ZDY0N2RlZjMtYWRlMi00MWY1LThkMWUtZmQ5YjA5NTE5ZTE4LmZkMDIxODhlLWM3NmEtNDE2YS04MWViLWZjYjBlMTE0NjFjY3gxNzMyNTQ2OTg1OTg2ODc5MTA3"
업로드 실패: Training/01.원천데이터/TS_OBJ_1/APT_FP_OBJ_037344144.PNG


파일 추출 중:   0%|          | 0/2041 [00:00<?, ?it/s]

Upload error: Connection was closed before we received a valid response from endpoint URL: "http://175.214.62.133:50008/tada/Training/01.%EC%9B%90%EC%B2%9C%EB%8D%B0%EC%9D%B4%ED%84%B0/TS_OBJ_2/APT_FP_OBJ_985691754.PNG?uploadId=ZDY0N2RlZjMtYWRlMi00MWY1LThkMWUtZmQ5YjA5NTE5ZTE4LjUzOTAwOTJjLTMyOTEtNGEwMC1hMDE3LTNiMWZkYzQ0OTBlNngxNzMyNTYyNjk0MjUzMDYwNzgy&partNumber=1".
업로드 실패: Training/01.원천데이터/TS_OBJ_2/APT_FP_OBJ_985691754.PNG
Upload error: Connection was closed before we received a valid response from endpoint URL: "http://175.214.62.133:50008/tada".
업로드 실패: Training/01.원천데이터/TS_OBJ_2/APT_FP_OBJ_983508955.PNG
Upload error: Connection was closed before we received a valid response from endpoint URL: "http://175.214.62.133:50008/tada".
업로드 실패: Training/01.원천데이터/TS_OBJ_2/APT_FP_OBJ_983560055.PNG
Upload error: Connection was closed before we received a valid response from endpoint URL: "http://175.214.62.133:50008/tada".
업로드 실패: Training/01.원천데이터/TS_OBJ_2/APT_FP_OBJ_983848789.PNG
Upload error:

KeyboardInterrupt: 

In [ ]:
# 최종 상태 확인  
db = Database(  
    host=os.getenv('DB_HOST'),  
    port=int(os.getenv('DB_PORT')),  
    user=os.getenv('DB_USER'),  
    password=os.getenv('DB_PASSWORD'),  
    database=os.getenv('DB_NAME')  
)  

query = "SELECT COUNT(*) as count FROM object_storage"  
result = db.getter(query)  
print(f"\n총 업로드된 파일 수: {result[0]['count']}")  

# 카테고리별 통계 확인  
query = """  
SELECT category, COUNT(*) as count   
FROM object_storage   
GROUP BY category   
ORDER BY count DESC  
"""  
records = db.getter(query)  
print("\n카테고리별 파일 수:")  
for record in records:  
    print(f"- {record['category']}: {record['count']}개")  